# Step-by-step Tutorial on Implementing the Dijkastra Algorithm in Python

## Why learn the Dijkastra's algorithm

Dijkastra's is the go-to algorithm for finding the shortest path between two points in a network, which has many applications. Examples include:

- GPS navigation systems finding the fastest route
- Routing data packets in computer networks
- Delivery services optimizing routes for efficiency

It's a fundamental algorithm in computer science and graph theory. Understanding and learning to implement it opens doors to more advanced graph algorithms and applications. It also teaches a valuable problem-solving approach through its greedy algorithm, which involves making the optimal choice at each step based on current information. This skill is transferable to other optimization algorithms. Dijkastra's algorithm may not be the most efficient in all scenarios but it can be a good baseline when solving "shortest distance" problems. 

Beyond just finding the shortest physical distance, it can also be used in cases such as:
- Social networks (suggesting connections)
- Finance (finding optimal investment paths)
- Project management (finding the most efficient workflow)

Even if you aren't interested in any of these things, implementing Dijkastra's algorithm in Python will teach you valuable skills as we will use a combination of:
- Python classes
- Dictionary and list comprehensions
- Priority queues
- The `networkx` library for graph manipulation
- Creating animations as GIFs

This will be a fun tutorial, so let's get started!

## Key concepts related to graphs

## The Dijkastra algorithm explained visually

## Step-by-step implementation of the Dijkastra algorithm in Python

### 0. Understanding graphs as dictionaries

### 1. Using a dictionary to store distances from source

### 2. Understanding a priority queue

### 3. Creating a class to represent a graph

### 4. The Dijkastra algorithm as a `while` loop

### 5. Verifying the results with `networkx`

## Visualizing graphs using Matplotlib and NetworkX

## Animating the Dijkastra's algorithm

## Conclusion